In [1]:
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.preprocessing import StandardScaler
from xgboost import plot_importance
import matplotlib.pyplot as plt
import xgboost as xgb
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)

In [2]:
identifier_name = 'flight_id'

most_important_features_names = [
     'wtc',
     'flown_distance',
     'aircraft_type',
     'longitude_max',
     'altitude_median',
     'groundspeed_max',
     'airline',
     'groundspeed_75percentile',
     'altitude_25percentile',
     'flight_duration',
     'latitude_min',
     'vertical_rate_std',
     'altitude_75percentile',
     'longitude_median',
     'longitude_std',
     'vertical_rate_25percentile',
     'longitude_min',
     'longitude_mean',
     'adep',
     'vertical_rate_max',
     'ades',
     'latitude_std',
     'latitude_max',
     'longitude_25percentile',
     'altitude_mean',
     'latitude_mean',
     'vertical_rate_75percentile',
     'latitude_median',
     'groundspeed_min',
     'country_code_adep',
     'country_code_ades',
     'latitude_25percentile',
     'longitude_count',
     'groundspeed_25percentile',
     'vertical_rate_min',
     'longitude_75percentile',
     'track_75percentile',
     'taxiout_time',
     'track_median',
     'vertical_rate_median',
     'latitude_75percentile',
     'track_25percentile',
     'month_day',
     'latitude_count',
     'altitude_std',
     'arrival_time_hour',
     'track_mean',
     'arrival_time_hour_minute',
     'vertical_rate_mean'
]

target_name = 'tow'

global_random_state = 123

In [3]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [4]:
def evaluate_model(model, data_df, features_columns, target_column):
    k_fold_number = 1
    features = data_df[features_columns]
    target = data_df[target_column]
    X_array = features.values
    y_array = target.values
    rmse_scores = []
    kf = KFold(n_splits=3, shuffle=True, random_state=global_random_state)
    for train_index, test_index in kf.split(X_array):
        print("Evaluating k-fold number: ", k_fold_number)
        X_train, X_test = features.iloc[train_index], features.iloc[test_index]
        y_train, y_test = target.iloc[train_index], target.iloc[test_index]
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        rmse_scores.append(rmse(y_test, y_pred))
        k_fold_number += 1
    return np.mean(rmse_scores)

In [5]:
encoded_challenge_set = pd.read_csv('data/encoded_challenge_set.csv')

In [6]:
def calculate_stat(
    dataframe: pd.DataFrame, 
    group_by_column: str, 
    target_column: str,
    stat_type: str
) -> pd.Series | None:
    group = dataframe.groupby(group_by_column)[target_column]
    if stat_type == 'count':
        return group.count()
    elif stat_type == 'max':
        return group.max()
    elif stat_type == 'min':
        return group.min()
    elif stat_type == '1percentile':
        return group.quantile(0.01)
    elif stat_type == '2percentile':
        return group.quantile(0.02)
    elif stat_type == '3percentile':
        return group.quantile(0.03)
    elif stat_type == '4percentile':
        return group.quantile(0.04)
    elif stat_type == '5percentile':
        return group.quantile(0.05)
    elif stat_type == '6percentile':
        return group.quantile(0.06)
    elif stat_type == '7percentile':
        return group.quantile(0.07)
    elif stat_type == '8percentile':
        return group.quantile(0.08)
    elif stat_type == '9percentile':
        return group.quantile(0.09)
    elif stat_type == '10percentile':
        return group.quantile(0.10)
    elif stat_type == '90percentile':
        return group.quantile(0.90)
    elif stat_type == '91percentile':
        return group.quantile(0.91)
    elif stat_type == '92percentile':
        return group.quantile(0.92)
    elif stat_type == '93percentile':
        return group.quantile(0.93)
    elif stat_type == '94percentile':
        return group.quantile(0.94)
    elif stat_type == '95percentile':
        return group.quantile(0.95)
    elif stat_type == '96percentile':
        return group.quantile(0.96)
    elif stat_type == '97percentile':
        return group.quantile(0.97)
    elif stat_type == '98percentile':
        return group.quantile(0.98)
    elif stat_type == '99percentile':
        return group.quantile(0.99)
        
    return None

In [7]:
def summarize_by_aircraft_type(
    data_df: pd.DataFrame,
    stat_types: list
) -> dict[int, dict]:
    stats_by_aircraft_df = {}
    for aircraft_type in sorted(list(data_df['aircraft_type'].unique())):
        stats_by_aircraft_df[aircraft_type] = {}

    for stat_type in stat_types:
        stat_by_aircraft = calculate_stat(dataframe=data_df, group_by_column='aircraft_type', target_column='tow', stat_type=stat_type)
        for i, value in enumerate(stat_by_aircraft):
            stats_by_aircraft_df[stat_by_aircraft.index[i]][stat_type] = value
    
    return stats_by_aircraft_df

In [8]:
stats_by_aircraft_df = summarize_by_aircraft_type(
    data_df = encoded_challenge_set[most_important_features_names+[target_name]],
    stat_types = [
        'count', 'min', 'max', 
        '1percentile', '2percentile', '3percentile', '4percentile', '5percentile', 
        '6percentile', '7percentile', '8percentile', '9percentile', '10percentile',
        '90percentile', '91percentile', '92percentile', '93percentile', '94percentile', '95percentile', 
        '96percentile', '97percentile', '98percentile', '99percentile']
)

total_count_before_summarization = len(encoded_challenge_set)
print(f"{total_count_before_summarization = }")

total_count_after_summarization = 0
for aircraft_type, aircraft_stats in stats_by_aircraft_df.items():
    total_count_after_summarization += aircraft_stats['count']
print(f"{total_count_after_summarization = }")

total_count_before_summarization = 369013
total_count_after_summarization = 369013


In [9]:
def evaluate_model_with_limit(model, data_df, features_columns, target_column, stats_by_aircraft_df) -> dict:
    
    features = data_df[features_columns]
    target = data_df[target_column]
    X_array = features.values
    y_array = target.values

    rmse_scores = {
        'prediction_without_margin': [],
        'min_max_margin': [],
        '1_percent_margin': [],
        '2_percent_margin': [],
        '3_percent_margin': [],
        '4_percent_margin': [],
        '5_percent_margin': [],
        '6_percent_margin': [],
        '7_percent_margin': [],
        '8_percent_margin': [],
        '9_percent_margin': [],
        '10_percent_margin': [],
    }
    
    kf = KFold(n_splits=3, shuffle=True, random_state=global_random_state)
    k_fold_number = 1
    for train_index, test_index in kf.split(X_array):
        print("Evaluating k-fold number: ", k_fold_number)

        k_fold_limited_prediction = {
            'prediction_without_margin': [],
            'min_max_margin': [],
            '1_percent_margin': [],
            '2_percent_margin': [],
            '3_percent_margin': [],
            '4_percent_margin': [],
            '5_percent_margin': [],
            '6_percent_margin': [],
            '7_percent_margin': [],
            '8_percent_margin': [],
            '9_percent_margin': [],
            '10_percent_margin': [],
        }
        
        X_train, X_test = features.iloc[train_index], features.iloc[test_index]
        y_train, y_test = target.iloc[train_index], target.iloc[test_index]
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        for i, aircraft_type in enumerate(X_test['aircraft_type']):
            predicted_value = y_pred[i]

            # Correct for min and max per aircraft type
            min_limit = stats_by_aircraft_df[aircraft_type]['min']
            max_limit = stats_by_aircraft_df[aircraft_type]['max']
            if predicted_value < min_limit:
                k_fold_limited_prediction['min_max_margin'].append(min_limit)
            elif predicted_value > max_limit:
                k_fold_limited_prediction['min_max_margin'].append(max_limit)
            else:
                k_fold_limited_prediction['min_max_margin'].append(predicted_value)

            # Correct for percentile margin per aircraft type between 1% and 10% by 1% increase
            for margin in range(1, 11, 1):
                min_percentile_limit = stats_by_aircraft_df[aircraft_type][f'{str(margin)}percentile']
                max_percentile_limit = stats_by_aircraft_df[aircraft_type][f'{str(100-margin)}percentile']
                if predicted_value < min_percentile_limit:
                    k_fold_limited_prediction[f'{str(margin)}_percent_margin'].append(min_percentile_limit)
                elif predicted_value > max_percentile_limit:
                    k_fold_limited_prediction[f'{str(margin)}_percent_margin'].append(max_percentile_limit)
                else:
                    k_fold_limited_prediction[f'{str(margin)}_percent_margin'].append(predicted_value)

        rmse_scores['prediction_without_margin'].append(rmse(y_test, y_pred))
        rmse_scores['min_max_margin'].append(rmse(y_test, np.array(k_fold_limited_prediction['min_max_margin'])))
        rmse_scores['1_percent_margin'].append(rmse(y_test, np.array(k_fold_limited_prediction['1_percent_margin'])))
        rmse_scores['2_percent_margin'].append(rmse(y_test, np.array(k_fold_limited_prediction['2_percent_margin'])))
        rmse_scores['3_percent_margin'].append(rmse(y_test, np.array(k_fold_limited_prediction['3_percent_margin'])))
        rmse_scores['4_percent_margin'].append(rmse(y_test, np.array(k_fold_limited_prediction['4_percent_margin'])))
        rmse_scores['5_percent_margin'].append(rmse(y_test, np.array(k_fold_limited_prediction['5_percent_margin'])))
        rmse_scores['6_percent_margin'].append(rmse(y_test, np.array(k_fold_limited_prediction['6_percent_margin'])))
        rmse_scores['7_percent_margin'].append(rmse(y_test, np.array(k_fold_limited_prediction['7_percent_margin'])))
        rmse_scores['8_percent_margin'].append(rmse(y_test, np.array(k_fold_limited_prediction['8_percent_margin'])))
        rmse_scores['9_percent_margin'].append(rmse(y_test, np.array(k_fold_limited_prediction['9_percent_margin'])))
        rmse_scores['10_percent_margin'].append(rmse(y_test, np.array(k_fold_limited_prediction['10_percent_margin'])))
        
        k_fold_number += 1
        
    return rmse_scores

In [10]:
xgb_model = xgb.XGBRegressor(
    max_depth=9,
    n_estimators=850,
    min_child_weight=9,
    learning_rate=0.2,
    subsample=1.0,
    colsample_bytree=0.8,
    objective='reg:squarederror', 
    eval_metric='rmse',
    random_state=global_random_state
)

rmse_scores = evaluate_model_with_limit(
    model=xgb_model, 
    data_df=encoded_challenge_set, 
    features_columns=most_important_features_names, 
    target_column=target_name,
    stats_by_aircraft_df=stats_by_aircraft_df
)

Evaluating k-fold number:  1
Evaluating k-fold number:  2
Evaluating k-fold number:  3


In [11]:
for test_margin, value in rmse_scores.items():
    print(f"{test_margin}: Mean RMSE Score: {np.mean(value)}, RMSE k-fold scores: {value}")

prediction_without_margin: Mean RMSE Score: 2858.3085356585557, RMSE k-fold scores: [2871.829262751778, 2857.684276701903, 2845.412067521986]
min_max_margin: Mean RMSE Score: 2841.5308332627897, RMSE k-fold scores: [2843.7305457794178, 2848.8712088949533, 2831.9907451139975]
1_percent_margin: Mean RMSE Score: 2840.7812322558652, RMSE k-fold scores: [2842.9414704271994, 2849.00680144589, 2830.3954248945065]
2_percent_margin: Mean RMSE Score: 2865.1063348893927, RMSE k-fold scores: [2866.7469769114878, 2876.6372341932633, 2851.934793563428]
3_percent_margin: Mean RMSE Score: 2902.7619073706687, RMSE k-fold scores: [2902.3745565354093, 2917.346941300883, 2888.564224275716]
4_percent_margin: Mean RMSE Score: 2949.241111311474, RMSE k-fold scores: [2947.814304021081, 2964.7796723849337, 2935.1293575284076]
5_percent_margin: Mean RMSE Score: 3018.1812647980555, RMSE k-fold scores: [3015.0910834488327, 3034.0134540521394, 3005.439256893194]
6_percent_margin: Mean RMSE Score: 3098.635372761448